In [ ]:
# install dependencies
!pip install --quiet google-cloud-storage
!pip install --quiet openpyxl
!pip install --quiet opencv-python==4.3.0.36

In [ ]:
import pandas as pd
import matplotlib
import cv2
import os
from datetime import datetime
from google.cloud import storage

In [ ]:
# read excel file from bucket
bucket_name = 'automated_angle_detection'
filepath = "gs://automated_angle_detection/test_video_data.xlsx"

In [ ]:
# video = "https://storage.cloud.google.com/bkt-mets-d-cc212-cmucapstone2022-mlb/VID2022/06/15/0060170/0060170L/0001A.mp4"
# media = blob_metadata('automated_angle_detection','VID2022_06_15_0060170_0060170L_0001A.mp4')
# first_part = media.split('?')[0]
# print(first_part)
# extract_images('tmp/0060170L/0001A.mp4', 'LHH', f)
# # https://storage.cloud.google.com/automated_angle_detection/VID2022_06_15_0060170_0060170L_0001A.mp4
# # cam = cv2.VideoCapture(video)
# # frames = cam.get(cv2.CAP_PROP_FPS)
# # print(cam.read())

In [ ]:
df = pd.read_excel(filepath, sheet_name='Sheet1')

In [ ]:
def extract_images(path, angle, f, folder):
        '''
        Read video path
        Extract frames from 2-6 seconds
        write angle and image path into a csv file
        '''
        location = path.split('/')
        path = 'tmp/'+location[11]+'/'+location[12]
        # print(path)
        cam = cv2.VideoCapture(path)
        cam.set(cv2.CAP_PROP_POS_MSEC,2000)   

        try:
            # creating a folder named data
            if not os.path.exists('data'):
                os.makedirs('data')
        
        # if not created then raise error
        except OSError:
            print ('Error: Creating directory of data')
        
        # frame
        currentframe = 0
        time_skips = float(500)
        totalTime = 0 
    
        #capturing 2-8 seconds of videos to extract images
        while(totalTime < 6000):
            
            # reading from frame
            ret,frame = cam.read()
            # print(ret,frame)
            # add training folder for each training
        
            if ret:
                # if video is still left continue creating images
                # replace space with '_'
                angle = angle.replace(" ", "_")
                
                filename = angle+'_'+str(int(datetime.timestamp(datetime.now()))) +str(currentframe)+ '.jpeg'
                name = './data/' + filename #path where file is saved
                
                gcspath = 'gs://automated_angle_detection/images/'+filename
                f.write(gcspath + ','+angle+'\n')
                
                #skipping by 0.5 seconds
                totalTime = (currentframe + 1)*time_skips
                cam.set(cv2.CAP_PROP_POS_MSEC,(totalTime)) 
                # print ('Creating...' + name)
        
                # writing the extracted images
                cv2.imwrite(name, frame, [cv2.IMWRITE_JPEG_QUALITY, 30]) #max size - 100KB (compression)
        
                # increasing counter so that it will
                # show how many frames are created
                currentframe += 1
                # print(currentframe)
                upload_blob(bucket_name, name, 'images/'+filename)
            else:
                break
        
        # Release all space and windows once done
        cam.release()
        cv2.destroyAllWindows()

In [ ]:
# upload image data to cloud bucket for creating a dataset
def upload_blob(bucket_name, source_file_name, destination_blob_name):    

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    
    generation_match_precondition = 0
    blob.upload_from_filename(source_file_name, if_generation_match=generation_match_precondition)
    
    print(f"File {source_file_name} uploaded to {destination_blob_name}.")

In [ ]:
# create a csv file for dataset creation and store it on gcp bucket
f = open('test_angle_image_data.csv','w')

In [ ]:
df.apply(lambda x: extract_images(x['path'], x['camera_angle'], f), axis=1)
f.close()

In [ ]:
bucket_name = 'automated_angle_detection'
file_name = 'test_angle_image_data.csv'

client = storage.Client()

bucket = client.get_bucket(bucket_name)

blob = bucket.blob(file_name)

with open(file_name, 'rb') as f:
    blob.upload_from_file(f)